# **멀티모달 LLM**

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

## **1. 멀티모달**

### **멀티모달이란?**

- **모달(Modal)** 이란?
    - **AI나 인지과학**에서 “모달(modal)”은 정보가 표현되거나 감지되는 형태/형식을 의미함 - **정보의 종류**/ **데이터의 형태**
    - 즉, **인간**이 세상을 인식할 때 사용하는 감각 채널(sensory modality) 과 같은 개념 - **시각(눈), 청각(귀), 촉각(피부) 등**

- **멀티모달(Multimodal)** 이란
    - <mark>**AI가 여러 형태(Modalities)의 정보를 동시에 이해하고 처리하는 기술**</mark>을 의미
    - 즉, 텍스트(text), 이미지(image), 음성(audio), 영상(video), 센서 데이터(sensor data) 등을 한 번에 다루어 서로 다른 형태의 데이터를 통합적으로 인식하고 추론할 수 있게 하는 것
    


### **데이터 모달리티 유형**

- **데이터 모달리티 유형**
|모달리티|설명|예시 응용|
|---|---|---|
|**텍스트(Text)**|자연어 문장, 문서, 코드 등|문서 분석, 챗봇, 번역|
|**이미지(Image)**|사진, 그림, 의료 영상 등|객체 인식, 의료 진단, 얼굴 인식|
|**오디오(Audio)**|음성, 음악, 환경 소음 등|음성 인식, 감정 분석, 음악 생성|
|**비디오(Video)**|동영상, 실시간 스트리밍|행동 인식, 비디오 요약, 감시 시스템|
|**센서 데이터**|GPS, LIDAR, IMU, 온도 등|자율주행, 로봇 제어, IoT 시스템|
|**깊이(Depth)**|3D 정보, 거리 측정|AR/VR, 3D 재구성, 공간 인식|


- **모달의 구성**
|구분|	내용|	예시|
|---|---|---|
|**단일모달 (Unimodal)**|	하나의 데이터 형태만 처리|	GPT-2 → 텍스트만 입력/출력|
|**멀티모달 (Multimodal)**|	여러 형태의 데이터를 동시에 처리|	GPT-4V → 텍스트 + 이미지 이해 가능|
|**크로스모달 (Cross-modal)**|	한 형태의 데이터를 다른 형태로 변환|	텍스트 설명 → 이미지 생성 (예: DALL·E, Midjourney)|


### **멀티모달 학습의 주요 과제**

|과제|설명|해결 방법|
|---|---|---|
|**표현(Representation)**|서로 다른 형식의 데이터를 공통 공간에 표현|Joint Embedding Space, Shared Representations|
|**정렬(Alignment)**|서로 다른 모달리티 간의 대응 관계 찾기|Cross-Modal Attention, Contrastive Learning|
|**융합(Fusion)** |여러 모달리티의 정보를 결합|Early Fusion, Late Fusion, Hybrid Fusion|
|**변환(Translation)**|한 모달리티에서 다른 모달리티로 변환|Image Captioning, Text-to-Image Generation|
|**공동학습(Co-learning)**|한 모달리티의 지식을 다른 모달리티 학습에 활용|Transfer Learning, Knowledge Distillation|


### **멀티모달의 발전**
- 인간의 인지능력은 시각, 청각, 언어를 함께 사용함.
- AI도 이를 모방하려면 “멀티모달 통합”이 필요했음

|시대|	주요 발전|	설명|
|---|---|---|
|**2012~2014**|	**CNN** 기반 이미지 인식 발전|	이미지 분류 정확도 급상승 (AlexNet, ResNet)|
|**2014~2015**|초기 멀티모달 연구|**Image Captioning**(이미지 설명 생성) 연구가 활발해졌습니다. Show and Tell(2015), Show, Attend and Tell(2016) 등의 모델이 등장|
|**2017~2019**|	**Transformer**와 **NLP 혁명**|	BERT, GPT 등으로 언어 이해 성능 향상|
|**2019~2021**|	**Vision-Language 모델** 발전|	**CLIP**(2021), DALL-E(2021) 등 이미지와 텍스트를 연결하는 모델이 등장함. **CLIP은 이미지-텍스트 쌍을 대조 학습(Contrastive Learning)으로 학습했음**
|**2022~2023**|**생성형 멀티모달 AI**|GPT-4(2023)가 멀티모달 기능을 갖추며 **이미지와 텍스트를 함께 처리할 수 있게 되었음**. Stable Diffusion, Midjourney 등 이미지 생성 모델도 대중화되었음|
|**2024~**|통합 멀티모달 시스템|Gemini 2.5, Claude 3.7, GPT-4o, Grok-4 등 **텍스트, 이미지, 오디오, 비디오를 모두 처리할 수 있는 통합 모델들이 등장**.2024년 멀티모달 AI 시장 규모는 16억 달러로 평가됨. 이후 계속 성장 중 |



---



## **2. 비전 트랜스포머(ViT)**

- **트랜스포머 기반 모델의 성공**
    - 분류와 클러스터링부터 검색과 생성 모델링에 이르기까지 다양한 언어 모델링 작업에 성공적으로 사용함
- **비전 트랜스포머**(**Vision Transformer, ViT**)
    - 트랜스포머의 성공을 컴퓨터 비전 분야로 일반화하는 방법을 찾으러는 노력(방법)
    - 종전의 합성곱 신경망(CNN)에 비해 이미지 인식 작업에서 매우 잘 동작한다는 것이 입증됨(An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale, 2020.10,  https://arxiv.org/abs/2010.11929)
    - **ViT**는 트랜스포머 구조의 핵심 구성 요소인 **인코더에 의존**함
    - **ViT**는 <mark>**이미지를 단어로 토큰화 하는 방법을 만들었음**</mark>
        - 텍스트를 토큰으로 분할하는 것처럼 원본 이미지를 이미지 패치(픽셀의 패치 patch)로 변환함
        - 이미지의 각 패치는 토큰과 달리 고유한 ID를 할당할 수 없음(텍스트 어휘사전에 있는 토큰과 달리 다른 이미지에 나타나는 경우가 거의 없기  때문)
        - 그 대신 **패치를 선형적으로 임베딩하여 수치 표현 임베딩을 만든다**.--> 이 임베딩이 인코더로 전달되면 텍스트 토큰처럼 처리되기 때문에 **텍스트와 이미지를 훈련하는 방식에 차이가 없음**


- **ViT 핵심 알고리즘**
<div style="display: flex; justify-content: space-around;">
  
  <img src="https://drive.google.com/uc?export=view&id=1W5eof7cmEJqqyeypnUOrski9TUwFIj31" width="48%" title="이미지: 핸즈온 LLM 1">

  <img src="https://drive.google.com/uc?export=view&id=1bDgXU5MQeMX95fnl5t4v_vFblg_Ezmmo" width="48%" title="이미지: 핸즈온 LLM">

</div>



### ✅ **예제 : ViT 모델을 이용하여 이미지 분류하기**

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
import torch
import matplotlib.pyplot as plt

# 모델과 전처리기 로드 (ViTImageProcessor 사용)
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")


# 이미지 로드
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/cats.png"
# url = "https://img.sbs.co.kr/newsnet/etv/upload/2022/10/07/30000795252_1280.jpg"
# url = "https://images.pexels.com/photos/34663263/pexels-photo-34663263.jpeg"

image = Image.open(requests.get(url, stream=True).raw)
# image = Image.open('dog.jpg')  # 이미지 파일 직접 사용

# 이미지를 RGB 모드로 명시적 변환 (중요!)
image = image.convert("RGB")
plt.imshow(image)
plt.axis('off')
plt.show()

# 이미지 전처리
inputs = processor(images=image, return_tensors="pt")

# 추론
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(-1).item()

    # 확률값도 함께 출력
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    confidence = probabilities[0][predicted_class].item()

print(f"예측된 클래스: {model.config.id2label[predicted_class]}")
print(f"확신도: {confidence:.2%}")


- **이미지 파일 읽어서 분류하기**

In [ ]:
# 이미지 로드
file = 'dog.jpg'
image = Image.open(file)

# 이미지를 RGB 모드로 명시적 변환 (중요!)
image = image.convert("RGB")
plt.imshow(image)
plt.axis('off')
plt.show()

# 이미지 전처리
inputs = processor(images=image, return_tensors="pt")

# 추론
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(-1).item()

    # 확률값도 함께 출력
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    confidence = probabilities[0][predicted_class].item()

print(f"예측된 클래스: {model.config.id2label[predicted_class]}")
print(f"확신도: {confidence:.2%}")


### ✅ **응용 : ViT 로 이미지 자동 태깅하기**

- 코드파일 :  _11_ml_멀티모달LLM_2_ViT로 이미지 자동 태깅하기_완성.ipynb




---



## **3. 멀티모달 임베딩 모델**

- **임베딩**
    - 임베딩 모델은 임베딩(또는 수치 표현)을 사용해 비슷한 문서를 찾고, 분류 작업을 적용하고, 토픽 모델일을 수행할 수 있었음
    - 임베딩은 LLM 애플리케이션을 구성하는데 핵심 역할을 수행함

- **멀티모달 임베딩 모델**
    - **텍스트와 시각 표현을 모두 포착할 수 있는 임베딩 모델**
    - 만들어진 임베딩이 동일한 벡터 공간에 놓이기 때문에 멀티모달 표현을 비교할 수 있다.
    - ex: 'pictures of a puppy' 텍스트와 가장 비슷한 이미지는? --> **질문과 관련성이 가장 높은 문서를 찾는 작업과 비슷**하다.
    - **CLIP(Contrastive Language-Image Pre-training)**
        - 많은 멀티모달 임베딩 모델중 가장 유명하고 널리 사용되는 모델


- **멀티모달 임베딩 벡터 공간**
<img src="https://drive.google.com/uc?export=view&id=1dlQtZ-ZblB2ZYYaylEkJ1jOIqGWp9Z8O" width="60%" title="이미지: 핸즈온 LLM">


### **CLIP 기반 모델**

- **CLIP**(Contrastive Language-Image Pre-training, 대조학습 언어-이미지 사전 훈련 모델)은 **이미지와 텍스트의 임베딩을 모두 계산할 수 있는 임베딩 모델임**
- **만들어진 임베딩은 동일한 벡터 공간에 놓임** --> 이미지 임베딩과 테스트 임베딩을 비교할 수 있다.

- **CLIP 모델이 유용한 작업**
    - <mark>**제로샷 분류**</mark> : 제로샷 이미지 분류는 **모델을 훈련하는 동안 이전에 본 적이 없는 클래스를 분류하는 작업**,
    - 이미지 임베딩을 후보 클래스 설명의 임베딩과 비교하여 가장 비슷한 클래스를 찾을 수 있음, https://huggingface.co/tasks/zero-shot-image-classification
    - **클러스터링** : 키워드가 어떤 이미지 집합에 속하는지 찾기 위해 이미지와 키워드 집합을 클러스터로 모음
    - **검색** : 수십억 개의 텍스트 또는 이미지에서 입력 텍스트나 이미지에 관련된 결과를 빠르게 찾을 수 있다.
    - **생성** : 이미지 생성(ex: 스테이블 디퓨전)에 멀티모달 임베딩을 사용함

### **CLIP이 멀티모달 임베딩을 생성하는 방법**

- ex: 수백만 개의 **이미지와 캡션으로 구성된 데이터셋 준비**
1. **이미지 인코더와 텍스트 인코더를 사용해 이미지와 텍스트를 각각 임베딩함** -> 임베딩 생성
2. **코사인 유사도를 사용해 문장 임베딩과 이미지 임베딩 사이의 유사도를 계산**
    - 훈련하는 동안 두 임베딩 사이의 유사도를 최적화함.
    - 비슷한 이미지/캡션 쌍에서는 유사도를 최대화하고 비슷하지 않은 이미지/캡션 쌍에서는 최소화함
3. **기대하는 유사도와 일치하도록 텍스트 인코더와 이미지 인코더가 업데이트됨**. 입력이 비슷하다면 벡터 공간에서 가까이 위치하도록 임베딩을 업데이트하게 됨
    - 이 과정을 **대조학습(contrastrive learning**)이라고 부른다.
    - 가능한 한 정확한 표현을 만들기 위해 **관련 없는 이미지와 캡션을 사용하는 네커티브 샘플이 훈련 과정에 포함됨**
    - 유사도 모델일을 통해 **무엇이 서로 비슷하게 만드는지뿐만 아니라 무엇이 서로 다르게 만드는지도 배움**

- **멀티모달 임베딩 모델 훈련을 위한 데이터 유형**
<img src="https://drive.google.com/uc?export=view&id=1MXRPynisuDNVzogq6rCBchZVhjWieLnb" width="60%" title="이미지: 핸즈온 LLM">



- **CLIP이 멀티모달 임베딩을 생성하는 방법**
<img src="https://drive.google.com/uc?export=view&id=1UYqqnJbV9WhT_aJqVvLvb9_WbwvqzH8p" width="65%" title="이미지: 핸즈온 LLM">


#### ✅ **예제 : OpenCLIP 모델로 이미지와 캡션 유사도 확인하는 과정 살펴보기**

1. **데이터 준비**

In [ ]:
from urllib.request import urlopen
from PIL import Image

# (스테이블 디퓨전)AI로 생성한 이미지를 로드합니다.
puppy_path = "https://bit.ly/4jYqmPu"   # 강아지 이미지 path
image = Image.open(urlopen(puppy_path)).convert("RGB")
caption = "a puppy playing in the snow"
image.size

In [ ]:
image

2. **OpenCLIP 임베딩 모델 사용하여 이미지& 텍스트 임베딩하기**
    - CLIP의 오픈 소스 버전 모델 사용
    - **OpenCLIP 모델**: **openai/clip-vit-base-patch32**   
        - **base**: 모델의 크기, 보통 'Small', 'Base', 'Large' 등으로 구분
        - **patch32** : 이미지를 32x32 픽셀 크기의 '패치(Patch)' 조각들로 나누어 입력으로 사용
        - 이 모델은 **제로샷 이미지 분류(Zero-Shot Image Classification**)에 가장 널리 사용됨
        - 모델에게 특정 이미지 한 장과 ["강아지 사진", "고양이 사진", "자동차 사진"] 같은 텍스트 후보들을 주면, 모델이 해당 이미지를 학습한 적이 없어도 어떤 텍스트 설명과 가장 유사한지 알려줄 수 있음

In [ ]:
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel

model_id = "openai/clip-vit-base-patch32"

# 텍스트 전처리를 위한 토크나이저를 로드합니다.
clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_id)

# 이미지 전처리를 위한 전처리기를 로드합니다.
clip_processor = CLIPProcessor.from_pretrained(model_id)

# 텍스트 임베딩과 이미지 임베딩을 생성하기 위한 메인 모델
model = CLIPModel.from_pretrained(model_id)

In [ ]:
# 입력을 토큰으로 나눕니다.
inputs = clip_tokenizer(caption, return_tensors="pt")
inputs

In [ ]:
# 입력 아이디를 토큰으로 되돌립니다.
clip_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
# 텍스트 임베딩을 만듭니다.
text_embedding = model.get_text_features(**inputs)
text_embedding.shape

In [ ]:
# 이미지를 전처리합니다.
processed_image = clip_processor(
    text=None, images=image, return_tensors='pt'
)['pixel_values']

# 모델 입력은 보통 [Batch, Channel, Height, Width]
# (e.g., [1, 3, 224, 224]) 형태
processed_image.shape

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# 시각화를 이해 이미지를 준비합니다.
img = processed_image.squeeze(0)    # 맨 앞의 'Batch' 차원(크기가 1)을 제거
print(img.shape)
img = img.permute(*torch.arange(img.ndim - 1, -1, -1)) # torch.arange(2, -1, -1)과 같으며, [2, 1, 0] 텐서를 생성
print(img.shape)
img = np.einsum('ijk->jik', img)    # NumPy의 아인슈타인 합(einsum) 표기법을 사용해 차원을 재정렬
print(img.shape)                    # Matplotlib이 요구하는 (Height, Width, Channel) 형식이 완성

# 전처리된 이미지를 출력합니다.
plt.imshow(img)
plt.axis('off')

In [ ]:
# 이미지 임베딩을 만듭니다.
image_embedding = model.get_image_features(processed_image)
image_embedding.shape

In [ ]:
# --- 1. PyTorch 텐서 상태에서 정규화를 수행합니다. ---
# (NumPy 함수가 아닌, PyTorch의 .norm() 함수로 통일합니다)
text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
image_embedding /= image_embedding.norm(dim=-1, keepdim=True)

# --- 2. 유사도 계산을 위해 텐서를 NumPy 배열로 변환합니다. ---
# 이 시점에 .detach().cpu().numpy()를 딱 한 번만 사용합니다.
text_embedding_np = text_embedding.detach().cpu().numpy()
image_embedding_np = image_embedding.detach().cpu().numpy()

# --- 3. NumPy 배열로 유사도(내적)를 계산합니다. ---
score = text_embedding_np @ image_embedding_np.T
score

# array([[0.33149645]], dtype=float32)


#### **[참고] 여러 이미지로 유사도 점수 계산하기**
- 앞에서 나온 유사도 점수(array([[0.33149645]], dtype=float32) )의 의미 파악을 위해 다른 이미지와 캡션으로 비교해본다.
- 사용 이미지 3개 : puppy, cat, car

In [ ]:
from urllib.request import urlopen
from PIL import Image

# AI로 생성한 이미지를 로드합니다.
cat_path = "https://bit.ly/42UJXJu"     # 고양이 이미지 path
car_path = "https://bit.ly/4cR4rHs"     # 자동차 이미지 path
paths = [puppy_path, cat_path, car_path]
images = [Image.open(urlopen(path)).convert("RGBA") for path in paths]
captions = [
    "a puppy playing in the snow",
    "a pixelated image of a cute cat",
    "A supercar on the road \nwith the sunset in the background"
]

import numpy as np

# 모든 이미지의 임베딩을 만듭니다.
image_embeddings = []
for image in images:
  image_processed = clip_processor(images=image, return_tensors='pt')['pixel_values']
  image_embedding = model.get_image_features(image_processed).detach().cpu().numpy()[0]
  image_embeddings.append(image_embedding)
image_embeddings = np.array(image_embeddings)

# 모든 캡션의 임베딩을 만듭니다.
text_embeddings = []
for caption in captions:
  inputs = clip_tokenizer(caption, return_tensors="pt")
  text_emb = model.get_text_features(**inputs).detach().cpu().numpy()[0]
  text_embeddings.append(text_emb)
text_embeddings = np.array(text_embeddings)

In [ ]:
# 이미지와 캡션의 코사인 유사도를 계산합니다.
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(image_embeddings, text_embeddings)
sim_matrix

In [ ]:
# 피겨 객체를 만듭니다.
def visualize_figure_matrix(sim_matrix, images, captions):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(14, 10))
    plt.imshow(sim_matrix, cmap='viridis')

    # 정답 레이블로 틱을 설정합니다.
    plt.yticks(range(len(captions)), captions, fontsize=18)
    plt.xticks([])

    # 이미지를 출력합니다.
    for i, image in enumerate(images):
        plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")

    # 캡션을 추가합니다.
    for x in range(sim_matrix.shape[1]):
        for y in range(sim_matrix.shape[0]):
            plt.text(x, y, f"{sim_matrix[y, x]:.2f}", ha="center", va="center", size=30)

    # 불필요한 요소를 제거합니다.
    for side in ["left", "top", "right", "bottom"]:
        plt.gca().spines[side].set_visible(False)

        # 크기를 조정합니다.
        plt.xlim([-0.5, len(captions) - 0.5])
        plt.ylim([len(captions) + 0.5, -2])
        # plt.title("Similarity Matrix", size=20)
        plt.savefig("sim_matrix.png", dpi=300, bbox_inches='tight')

visualize_figure_matrix(sim_matrix, images, captions)

#### ✅ **예제: SBERT 임베딩 모델 방식 사용**

- **임베딩을 쉽게 만들 수 있는 CLIP기반 모델 사용해보기**
- **OpenCLIP 모델**: **clip-ViT-B-32**
    - OpenAI가 2021년 논문 "Learning Transferable Visual Models From Natural Language Supervision"에서 사용한 공식 모델 아키텍처 명칭
    - <mark>'멀티모달 AI'와 '제로샷 학습'을 설명할 때 가장 기본이 되고 상징적인 모델</mark>
    - 앞에서 사용한 **openai/clip-vit-base-patch32** 와 동일한 모델
        - 허깅페이스에서 사용할 때는 openai/clip-vit-base-patch32 로 등록되어 있음

In [ ]:
from sentence_transformers import SentenceTransformer, util

# SBERT 호환 CLIP 모델을 로드합니다.
model = SentenceTransformer('clip-ViT-B-32')

# 이미지를 인코딩합니다.
image_embeddings = model.encode(images)

# 캡션을 인코딩합니다.
text_embeddings = model.encode(captions)

# 코사인 유사도를 계산합니다.
sim_matrix = util.cos_sim(image_embeddings, text_embeddings)
print(sim_matrix)

visualize_figure_matrix(sim_matrix, images, captions)



---



## **4. 텍스트 생성 모델을 멀티모달로 만들기**

- **텍스트 생성 모델**
    - <mark>텍스트 생성 모델의 경우 특정 입력 이미지에 대해 추론할 수 있다.</mark>
        - ex: 피자 이미지를 주고 어떤 재료가 들어 있는지 물을 수 있다
        - ex: 에펠탑 사진을 보여 주고 언제 지어졌는지 어디에 있는지 물어볼 수도 있다.
        

### **BLIP**

- **BLIP**
    - **BLIP** **Bootstrapping Language-Image Pre-training** for Unified Vision-Language Understanding and Generation
    - BLIP은 2022년 1월 Salesforce AI Research에서 발표한 논문으로, 당시 Vision-Language Pre-training (VLP) 분야의 두 가지 큰 문제를 해결하며 등장한 혁신적인 모델
    - **[BLIP의 핵심 목표]**
        1. **모델의 한계 (Understanding vs. Generation)**
        2. **데이터의 한계 (Noisy Web Data)**
        - **CapFilt** (**Captioning and Filtering**)라는 2단계 프로세스로 구현
            - Captioning : Captioner (캡션 생성기)
            - Filtering :  Filter (노이즈 제거기)
    - "하나의 모델 아키텍처로 '이해(Understanding)'와 '생성(Generation)'을 모두 잘하면서,
    - 동시에 '지저분한 웹 데이터(Noisy Data)'를 스스로 정제해서 학습 효율을 극대화하자!"
- **BLIP-1이 VLP 모델의 '체급(데이터+아키텍처)' 자체를 키우려 했다면**,
- **BLIP-2는 '효율성'에 초점을 맞춰 이미 존재하는 강력한 모듈들을 '조립'하는 방식으로 패러다임을 전환**

#### ✅ **예제 : BLIP으로 이미지 자동 캡셔닝하기**

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import matplotlib.pyplot as plt

# 모델 및 프로세서 로드
model_name = "Salesforce/blip-image-captioning-base"
blip_processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

# 이미지 입력 : 적절한 이미지 파일 사용
image = Image.open("dog.jpg")
# 이미지 시각화
plt.imshow(image)
plt.axis('off')
plt.show()


# 이미지 캡션 생성 (LM 기능 테스트)
# 조건 없이 생성 (Unconditional Captioning)
# 이미지만 입력으로 넣어줍니다.
inputs_uncond = blip_processor(images=image, return_tensors="pt")
out_uncond = model.generate(**inputs_uncond, max_new_tokens=50)
caption_uncond = blip_processor.decode(out_uncond[0], skip_special_tokens=True)

print("--- BLIP-1 이미지 캡셔닝 (조건 없음) ---")
print(f"생성된 캡션: {caption_uncond}")


In [ ]:
# 웹에서 이미지를 찾아서 입력해주세요.
image = Image.open("pexels-kylenehashimoto-1334441.jpg")
# 이미지 시각화
plt.imshow(image)
plt.axis('off')
plt.show()


# 이미지 캡션 생성 (LM 기능 테스트)
# 조건 없이 생성 (Unconditional Captioning)
# 이미지만 입력으로 넣어줍니다.
inputs_uncond = blip_processor(images=image, return_tensors="pt")
out_uncond = model.generate(**inputs_uncond, max_new_tokens=50)
caption_uncond = blip_processor.decode(out_uncond[0], skip_special_tokens=True)

print("--- BLIP-1 이미지 캡셔닝 (조건 없음) ---")
print(f"생성된 캡션: {caption_uncond}")


### **BLIP-2**

- 두 도메인 사이의 간극을 메꾸기 위해 기존 모델에 멀티모달 기능을 추가하려는 시도
- 핵심 아이디어는 **이미 잘 만들어진 강력한 비전 모델과 LLM을 '얼려서(frozen)' 재사용하고, 그 사이를 연결하는 '다리'만 잘 학습시키자**는 것

#### **모델 지정**

In [ ]:
# 모델 다운로드하는데 약간 시간이 걸린다.(T4 3~5분 )
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

# 전처리기와 메인 모델을 로드합니다.
# # Choose specific model because of: https://huggingface.co/Salesforce/blip2-opt-2.7b/discussions/39
model_name = "Salesforce/blip2-opt-2.7b"
blip_processor = AutoProcessor.from_pretrained(model_name)
model = Blip2ForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

# 추론 속도를 높이기 위해 모델을 GPU에 전송합니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

#### **이미지 전처리**

In [ ]:
from urllib.request import urlopen
from PIL import Image
import matplotlib.pyplot as plt

# 이미지를 로드합니다.
car_path = "https://bit.ly/4cR4rHs" # 수퍼카
image = Image.open(urlopen(car_path)).convert("RGB")

# 이미지 시각화
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
# 이미지를 전처리합니다.
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
inputs["pixel_values"].shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def show_image(image_inputs):
    # 넘파이 배열로 변환하고, 크기를 (1, 3, 224, 224)에서 (224, 224, 3)로 바꿉니다.
    image_inputs = inputs["pixel_values"][0].detach().cpu().numpy()
    image_inputs = np.einsum('ijk->kji', image_inputs)
    image_inputs = np.einsum('ijk->jik', image_inputs)

    # RGB 값에 해당하는 0-255 범위로 변환합니다.
    scaler = MinMaxScaler(feature_range=(0, 255))
    image_inputs = scaler.fit_transform(image_inputs.reshape(-1, image_inputs.shape[-1])).reshape(image_inputs.shape)
    image_inputs = np.array(image_inputs, dtype=np.uint8)

    # 넘파이 배열을 Image 객체로 바꿉니다.
    return Image.fromarray(image_inputs)

t_image = show_image(inputs)
t_image

In [ ]:
t_image.info

#### **텍스트 전처리**(skip 가능)

In [ ]:
blip_processor.tokenizer

In [ ]:
# 텍스트를 전처리합니다.
text = "Her vocalization was remarkably melodic"
token_ids = blip_processor(text=text, return_tensors="pt")
token_ids = token_ids.to(device, torch.float16)["input_ids"][0]

# 입력 ID를 토큰으로 되돌립니다.
tokens = blip_processor.tokenizer.convert_ids_to_tokens(token_ids)
tokens

In [ ]:
# 특수 토큰을 밑줄 문자로 바꿉니다.
tokens = [token.replace("Ġ", "_") for token in tokens]
tokens

#### ✅ **예제 : 이미지 캡셔닝**

In [ ]:
# AI가 생성한 수퍼카 이미지를 로드합니다.
image = Image.open(urlopen(car_path)).convert("RGB")

# 이미지를 전처리하여 입력을 준비합니다.
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
show_image(inputs)

In [ ]:
# 이미지를 임베딩을 만들고 Q-포머의 출력을 디코더(LLM)에 전달해 토큰 ID를 생성합니다.
generated_ids = model.generate(**inputs, max_new_tokens=20)

# 토큰 ID를 바탕으로 텍스트를 생성합니다.
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
generated_text
print(f"생성된 캡션: {generated_text}")

#### ✅ **예제 : 이미지 설명**(로르샤흐 테스트를 이용한 심리분석)
- [참고] https://blog.naver.com/eehome/220365180030



In [ ]:
# 로르샤흐 이미지를 로드합니다
url = 'https://postfiles.pstatic.net/20150520_195/eehome_1432100347266Kk6U5_JPEG/2009073100028_0.jpg?type=w3840'
image = Image.open(urlopen(url)).convert("RGB")
image

In [ ]:
# 로르샤흐 이미지를 로드합니다.
url = 'https://postfiles.pstatic.net/20150520_195/eehome_1432100347266Kk6U5_JPEG/2009073100028_0.jpg?type=w3840'
image = Image.open(urlopen(url)).convert("RGB")

# 캡션을 생성합니다.
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
generated_ids = model.generate(**inputs, max_new_tokens=20)
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
print(f"생성된 캡션: {generated_text}")

### ✅ **예제 : 채팅 기반 멀티모달 프롬프트**

In [ ]:
# AI로 생성한 수퍼카 이미지를 로드합니다.
image = Image.open(urlopen(car_path)).convert("RGB")

In [ ]:
# 시각 질문 답변
prompt = "Question: Write down what you see in this picture. Answer:"

# 이미지와 프롬프트를 모두 전처리합니다.
inputs = blip_processor(image, text=prompt, return_tensors="pt").to(device, torch.float16)

# 텍스트를 생성합니다.
generated_ids = model.generate(**inputs, max_new_tokens=30)
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
print(f"생성된 캡션: {generated_text}")

In [ ]:
# 채팅 스타일의 프롬프트
prompt = "Question: Write down what you see in this picture. Answer: A sports car driving on the road at sunset. Question: What would it cost me to drive that car? Answer:"

# 출력을 생성합니다.
inputs = blip_processor(image, text=prompt, return_tensors="pt").to(device, torch.float16)
generated_ids = model.generate(**inputs, max_new_tokens=30)
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
generated_text

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets

def text_eventhandler(*args):
  question = args[0]["new"]
  if question:
    args[0]["owner"].value = ""

    # 프롬프트를 만듭니다.
    if not memory:
      prompt = " Question: " + question + " Answer:"
    else:
      template = "Question: {} Answer: {}."
      prompt = " ".join(
          [
              template.format(memory[i][0], memory[i][1])
              for i in range(len(memory))
          ]
      ) + " Question: " + question + " Answer:"

    # 텍스트를 생성합니다.
    inputs = blip_processor(image, text=prompt, return_tensors="pt")
    inputs = inputs.to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=100)
    generated_text = blip_processor.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )
    generated_text = generated_text[0].strip().split("Answer: ")[-1]

    # 메모리를 업데이트합니다.
    memory.append((question, generated_text))

    # 출력에 할당합니다.
    output.append_display_data(HTML("<b>USER:</b> " + question))
    output.append_display_data(HTML("<b>BLIP-2:</b> " + generated_text))
    output.append_display_data(HTML("<br>"))

# 위젯을 준비합니다.
in_text = widgets.Text()
in_text.continuous_update = False
in_text.observe(text_eventhandler, "value")
output = widgets.Output()
memory = []

# 채팅 상자를 출력합니다.
display(
    widgets.VBox(
        children=[output, in_text],
        layout=widgets.Layout(display="inline-flex", flex_flow="column-reverse"),
    )
)